<a href="https://colab.research.google.com/github/CD-AC/DataEnginner-Gas_Station_Geolocation/blob/main/Dataset_Station.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import random
import pandas as pd
from tqdm import tqdm
from bs4  import BeautifulSoup
from cred_here import YOUR_API_KEY
import json
from tqdm import tqdm
import time

import ast

In [2]:
YOUR_API_KEY = YOUR_API_KEY

In [3]:
df = pd.read_csv('Dataset_Station.csv')

In [4]:
df.head(2)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado
0,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,GASOLINA CORRIENTE OXIGENADA,9860,A
1,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,BIODIESEL EXTRA,9780,A


# Clean Data

In [5]:
df[df['Periodo']==2022]['Mes'].max()

2

In [8]:
df2 =  df[(df['Periodo']==2022) & (df['Mes']==2)]
df2 =  df2[(df2['Departamento']=='ANTIOQUIA') | (df2['Departamento']=='BOGOTA D.C.')]

In [9]:
df2['Municipio'].value_counts().head(10)

BOGOTA  D.C.    857
Name: Municipio, dtype: int64

In [10]:
df3 = df2[(df2['Departamento']=='BOGOTA D.C.') | ((df2['Departamento']=='ANTIOQUIA') & ((df2['Municipio']=='MEDELLIN') | (df2['Municipio']=='ITAGUI') | (df2['Municipio']=='BELLO') | (df2['Municipio']=='SABANETA') | (df2['Municipio']=='RIONEGRO') | (df2['Municipio']=='ENVIGADO')))]
df3.reset_index(inplace = True)
df3.drop(columns = 'index',inplace = True)

In [ ]:
df3.head(2)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado
0,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,GASOLINA EXTRA OXIGENADA,17880.0,A
1,2022,6,5,ANTIOQUIA,5001,MEDELLIN,ESTACION DE SERVICIO TERPEL LA 45,TERPEL,CARRERA 45 No. 66-10,BIODIESEL EXTRA,9040.0,A


In [11]:
df3['Municipio'].unique()

array(['BOGOTA  D.C.'], dtype=object)

In [12]:
df3['Dir2'] = df3['Direccion'].apply(lambda x : x.replace('#','No'))
df3['Full_Address'] = df3.apply(lambda x : x['Direccion'] + ', ' + x['Municipio'].capitalize() + ', Colombia' , axis = 1)

In [13]:
df3['Full_Address'].nunique()

239

In [14]:
df3['Full_Address'][0]

'AVENIDA CARRERA 7 No. 123A-48, Bogota  d.c., Colombia'

In [15]:
estaciones =  df3[['Nombre_comercial','Direccion','Full_Address']].drop_duplicates(subset = 'Nombre_comercial')
estaciones.reset_index(inplace = True)
estaciones.drop(columns = 'index',inplace = True)

In [16]:
estaciones['Full_Address'][0]

'AVENIDA CARRERA 7 No. 123A-48, Bogota  d.c., Colombia'

In [18]:
address = estaciones['Full_Address'][0]
url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={YOUR_API_KEY}'
response = requests.get(url).json()

In [19]:
def Get_Coords(address,YOUR_API_KEY):

    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={YOUR_API_KEY}'

    try :

        response = requests.get(url).json()
        CleanAddress = response['items'][0]['title'].upper()
        LAT = response['items'][0]['position']['lat']
        LNG = response['items'][0]['position']['lng']

        results = [CleanAddress,LAT,LNG]

    except :

        results = ['Not Found','NA','NA']


    return results

In [20]:
Get_Coords(address,YOUR_API_KEY)

['Not Found', 'NA', 'NA']

# Extracción de coordenadas

In [21]:
coordenadas = []
tiempo = [1,2,2.5]

for i in tqdm(range(len(estaciones['Full_Address'])),colour = 'green'):

    try :

        R = Get_Coords(estaciones['Full_Address'][i],YOUR_API_KEY)
        geo_source = R[1],R[2]
        coordenadas.append(geo_source)

    except :

        coordenadas.append('Error')


    time.sleep(random.choices(tiempo)[0])



100%|██████████| 239/239 [08:15<00:00,  2.07s/it]


In [22]:
estaciones['Coords'] = coordenadas

In [23]:
len(df3)

857

In [24]:
estaciones.head(2)

,Nombre_comercial,Direccion,Full_Address,Coords
0,SUPER ESTACION DE SERVICIO TEXACO 10,AVENIDA CARRERA 7 No. 123A-48,"AVENIDA CARRERA 7 No. 123A-48, Bogota d.c., C...","(NA, NA)"
1,ESTACION DE SERVICIO ROXI,Carrera 24 No 7-11,"Carrera 24 No 7-11, Bogota d.c., Colombia","(NA, NA)"


In [ ]:
df_final = pd.merge(df3,estaciones[['Full_Address','Coords']],how = 'inner' , on = 'Full_Address')

In [ ]:
df_final.head(3)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado,Dir2,Full_Address,Coords
0,2022,6,5,ANTIOQUIA,5266,ENVIGADO,ESTACION DE SERVICIOS PREMIUM LA SEBASTIANA,TEXACO,CARRERA 27 No. 36 SUR-199,GASOLINA CORRIENTE OXIGENADA,9710.0,A,CARRERA 27 No. 36 SUR-199,"CARRERA 27 No. 36 SUR-199, Envigado, Colombia","(6.16364, -75.56929)"
1,2022,6,5,ANTIOQUIA,5266,ENVIGADO,ESTACION DE SERVICIOS PREMIUM LA SEBASTIANA,TEXACO,CARRERA 27 No. 36 SUR-199,BIODIESEL EXTRA,10510.0,A,CARRERA 27 No. 36 SUR-199,"CARRERA 27 No. 36 SUR-199, Envigado, Colombia","(6.16364, -75.56929)"
2,2022,6,5,ANTIOQUIA,5266,ENVIGADO,ESTACION DE SERVICIOS PREMIUM LA SEBASTIANA,TEXACO,CARRERA 27 No. 36 SUR-199,BIODIESEL EXTRA,10510.0,A,CARRERA 27 No. 36 SUR-199,"CARRERA 27 No. 36 SUR-199, Envigado, Colombia","(6.16364, -75.56929)"


In [ ]:
df_final = pd.merge(df3,estaciones[['Full_Address','Coords']],how = 'inner' , on = 'Full_Address')

In [ ]:
df_final.to_csv('DATASET_FINAL.CSV',index = False)